In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# File paths
guj_train = "/content/drive/MyDrive/Project_Glory/Guj_train.csv"
guj_test = "/content/drive/MyDrive/Project_Glory/Guj_val_article.csv"

# Install dependencies
!pip install transformers
!pip install pytorch-lightning

# Imports
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import MT5Tokenizer, MT5ForConditionalGeneration
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CSVs
df_train = pd.read_csv(guj_train)
df_test = pd.read_csv(guj_test)

# Ensure Summary column exists in test set
if 'Summary' not in df_test.columns:
    df_test['Summary'] = ""

# Clean training and test data
df_train = df_train[["Summary", "Article"]].dropna()
df_train = df_train[df_train["Summary"].str.strip() != ""]

df_test = df_test[["Summary", "Article"]].dropna()

# Split train/val
df_train, df_valid = train_test_split(df_train, test_size=0.1, random_state=42)

# Load tokenizer
MODEL_NAME = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token  # Important for mT5
tokenizer.padding_side = "right"

# Dataset
class NewsSummaryDataset(Dataset):
    def __init__(self, data, tokenizer, text_max_tokens_len=512, summary_max_tokens_len=75):
        self.data = data.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.text_max_tokens_len = text_max_tokens_len
        self.summary_max_tokens_len = summary_max_tokens_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = str(self.data.loc[idx, "Article"])
        summary = str(self.data.loc[idx, "Summary"])
        input_text = "summarize: " + article

        input_encoding = self.tokenizer(
            input_text,
            max_length=self.text_max_tokens_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            summary,
            max_length=self.summary_max_tokens_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        labels = target_encoding.input_ids.squeeze()

        # Ensure valid target
        if all(token == tokenizer.pad_token_id for token in labels.tolist()):
            labels[0] = tokenizer.eos_token_id
        labels[labels == tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_encoding.input_ids.squeeze(),
            "attention_mask": input_encoding.attention_mask.squeeze(),
            "labels": labels,
            "decoder_attention_mask": target_encoding.attention_mask.squeeze(),
        }

# DataModule
class NewsSummaryDataModule(pl.LightningDataModule):
    def __init__(self, train_df, val_df, tokenizer, batch_size=2, text_max_tokens_len=512, summary_max_tokens_len=75):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.text_max_tokens_len = text_max_tokens_len
        self.summary_max_tokens_len = summary_max_tokens_len

    def setup(self, stage=None):
        self.train_dataset = NewsSummaryDataset(self.train_df, self.tokenizer, self.text_max_tokens_len, self.summary_max_tokens_len)
        self.val_dataset = NewsSummaryDataset(self.val_df, self.tokenizer, self.text_max_tokens_len, self.summary_max_tokens_len)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=2)

# Lightning Model
class NewsSummaryModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = MT5ForConditionalGeneration.from_pretrained(MODEL_NAME)

    def forward(self, input_ids, attention_mask, labels, decoder_attention_mask=None):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask,
        )
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        loss, _ = self(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
            decoder_attention_mask=batch["decoder_attention_mask"],
        )
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, _ = self(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
            decoder_attention_mask=batch["decoder_attention_mask"],
        )
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=2e-5)

# Training
BATCH_SIZE = 16
EPOCHS = 8

datamodule = NewsSummaryDataModule(df_train, df_valid, tokenizer, batch_size=BATCH_SIZE)
model = NewsSummaryModel()

trainer = pl.Trainer(
    max_epochs=EPOCHS,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,
    precision="32",
    gradient_clip_val=1.0
)

trainer.fit(model, datamodule=datamodule)
trainer.validate(model, datamodule=datamodule)

# Inference
model.freeze()

def summarize_text(text, max_length=150):
    input_text = "summarize: " + text
    encoding = tokenizer(
        input_text,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    input_ids = encoding.input_ids.to(model.device)
    attention_mask = encoding.attention_mask.to(model.device)

    generated_ids = model.model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_beams=4,
        early_stopping=True,
        repetition_penalty=2.5,
        length_penalty=1.0,
        no_repeat_ngram_size=3
    )
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Try a sample
sample_text = df_valid.iloc[0]["Article"]
print("Original Text:\n", sample_text)
print("\nGenerated Summary:\n", summarize_text(sample_text))


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 65.8 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=8` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     1.287343978881836     │
└───────────────────────────┴───────────────────────────┘

Original Text:
 વડોદરા: વડોદરા શહેરમાં એક 19 વર્ષીય આશાસ્પદ ખેલાડીનું ડેંગ્યૂથી નિધન થયું છે. શહેરના આજવા રોડ ખાતે રહેતી 19 વર્ષીય સાક્ષી રાવલ (Sakshi Rawal)નું નિધન થયું છે. સાક્ષીએ વર્ષ 2019માં ઝારખંડ (Jharkhand) ખાતે યોજાયેલી જુડોની સુરાશ કોમ્પિટિશનમાં બ્રોન્ઝ મેડલ મેળવ્યો હતો. વડોદરા શહેર (Vadodara city)માં વકરેલા પાણીજન્ય રોગચાળાએ એક નેશનલ પ્લેયરનો ભોગ લીધો છે. નેશનલ પ્લેયર એવી સાક્ષીના નિધનથી પરિવારમાં શોક વ્યાપી ગયો છે. 19 વર્ષની ઉંમરે દીકરી ગુમાવતા પરિવારના સભ્યો ભાંગી પડ્યાં હતાં. અંતિમ સંસ્કાર વખતે હાજર તમામ લોકોની આંખમાં આંસુ જોવા મળ્યા હતા. પરિવારે આક્ષેપ લગાવ્યો હતો કે તંત્રના પાપે અમારે દીકરી ગુમાવવાનો વારો આવ્યો છે. ઉલ્લેખનીય છે કે વડોદરા શહેરમાં હાલ પાણીજન્ય રોગચાળાએ ભરડો લીધો છે. શહેરના અનેક વિસ્તારો રોગચાળાના ભરડવામાં આવી ગયા છે. શહેરમાં પાણીજન્ય રોગચાળાનો કહેર વડોદરા પાણીજન્ય રોગચાળાનો કહેર જોવા મળ્યો છે. છેલ્લા 15 દિવસમાં પાલિકાના ચોપડે ડેંગ્યૂના 83 અને ચિકનગુનિયાના 35 કેસ નોંધાયા છે. આ પહેલા પાણીજન્ય રોગચાળાના કારણે નાગરવાડામાં ત્રણ મહિલાનાં મોત થયા હતા, હવે નેશનલ 

In [2]:
sample_text = df_valid.iloc[10]["Article"]
print("Original Text:\n", sample_text)
print("\nGenerated Summary:\n", summarize_text(sample_text))

Original Text:
 ભરૂચ અને અંક્લેશ્વરને જોડતો નર્મદા મૈયા બ્રીજ બે શહેર વચ્ચેનું અંતર ઘટાડવા સાથે ટ્રાફિકની સમસ્યામાંથી મુક્ત અપાવનાર મહત્વનો બ્રીજ સાબિત થયો છે. જોકે, નર્મદા મૈયા બ્રીજના નિર્માણ બાદ આ બ્રીજ અકસ્માત ઝોન તેમજ સુસાઇડ પોઇન્ટ બની ગયો હોય તેમ લાગી રહ્યું છે. બ્રીજ પર પુરઝડપે જતાં વાહનો વચ્ચે અકસ્માતના બનાવો વધી ગયાં છે. બીજી તરફ નર્મદા મૈયા બ્રીજ પરથી છાસવારે લોકો આપઘાત કરવાના ઇરાદે નર્મદા નદીમાં ભુસકો મારતાં હોવાના બનાવો સામે આવી રહ્યાં છે. ત્યારે નર્મદા મૈયા બ્રીજ પરથી ભારદારી વાહનોની અવરજવર બંધ કરવા તેમજ પોલીસ દ્વારા બ્રીજ પર સતત પેટ્રોલિંગ કરવામાં આવે તેવી માંગ ઉઠી છે.અંકલેશ્વર અને ભરૂચ ને જોડાતા નર્મદા મૈયા બ્રિજ પર અચાનક રીક્ષા ઉભી થઇ જતાં કાર ધડાકાભેર ઘુસી જવા પામી હતી . 7 વ્યક્તિઓને ઇજા પહોંચી હતી. 108 ની મદદ થી ઇજાગ્રસ્તોને સારવાર અર્થે ખસેડાયો હતા. અકસ્માતના પગલે વાહન વ્યવહાર પર આંશિક અસર થઇ હતી.અંકલેશ્વરથી ભરૂચ તરફ જતી પેસેન્જર ભરેલી રીક્ષા નર્મદા મૈયા બ્રિજ પર એકાએક ઊભી કરી દેતા પાછળથી આવી રહેલ કાર રીક્ષા માં ધડાકાભેર ભટકાઈ ગયા હતા. કાર અને રીક્ષા વચ્ચે સર્જાયેલા 

In [5]:
!pip install evaluate


In [17]:
import gradio as gr

# --- Your summarization function (keep your model logic here) ---
def summarize_interface(article):
    summary = summarize_text(article)   # ✅ your model function
    return summary


# --- Custom colorful theme (only valid tokens) ---
theme = gr.themes.Base(
    primary_hue="purple",
    secondary_hue="yellow",
    neutral_hue="cyan"
).set(
    background_fill_primary="linear-gradient(to right, #ffe57f, #ffd180)",  # golden-orange gradient
    background_fill_secondary="linear-gradient(to right, #81d4fa, #b388ff)",  # blue-purple gradient
    input_background_fill="#ffffff",
    button_primary_background_fill="#ff6f00",  # vibrant orange
    button_primary_text_color="#ffffff",
    body_text_color="#263238"  # dark slate
)


# --- Launch Gradio Interface ---
gr.Interface(
    fn=summarize_interface,
    inputs=gr.Textbox(
        lines=12,
        label="📰 Gujarati Article",
        placeholder="લેખ અહીં પેસ્ટ કરો... (Paste article here)",
        elem_id="input-box"
    ),
    outputs=gr.Textbox(
        lines=6,
        label="🧠 Generated Summary",
        elem_id="output-box"
    ),
    title="🎉 Gujarati News Summarizer",
    description="""
    <div style='text-align: center; font-size: 16px; color: #37474f; font-family: Comic Sans MS;'>
        <p>🔍 <b>Paste your Gujarati news article</b> below and get a short, clear summary using a powerful mT5 model.</p>
        <p>✨ Built with <span style="color:#7e57c2;"><b>HuggingFace Transformers</b></span> and <span style="#ff6f00;"><b>PyTorch Lightning</b></span></p>
    </div>
    """,
    theme=theme,
    allow_flagging="never",
    live=True,
    css="""
        /* Global font override */
        * {
            font-family: "Comic Sans MS", cursive, sans-serif !important;
        }
        #input-box textarea {
            border: 2px solid #7e57c2;
            box-shadow: 0 0 10px rgba(126, 87, 194, 0.4);
            background: #fff8e1;
        }
        #output-box textarea {
            border: 2px solid #ff6f00;
            background: #e1f5fe;
            color: #263238;
            font-weight: bold;
        }
        .gr-button {
            font-size: 16px;
            font-weight: bold;
            border-radius: 10px;
            transition: 0.3s ease;
        }
        .gr-button:hover {
            background: #ffd54f !important;
            color: #000 !important;
        }
    """
).launch(share=True)


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d8788c01f537151c67.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
!pip install bert_score

In [13]:
import pandas as pd
from bert_score import score

# Index to test
index = 10

# Get article and reference summary
sample_text = df_valid.iloc[index]["Article"]

reference_summary = "ભરૂચ અને અંક્લેશ્વરને જોડતો નર્મદા મૈયા બ્રીજ બે શહેર વચ્ચેનું અંતર ઘટાડવા સાથે ટ્રાફિકની સમસ્યામાંથી મુક્ત અપાવનાર મહત્વનો બ્રીજ સાબિત થયો છે. જોકે, નર્મદા મૈયા બ્રીજના નિર્માણ બાદ આ બ્રીજ અકસ્માત ઝોન તેમજ સુસાઇડ પોઇન્ટ બની ગયો હોય તેમ લાગી રહ્યું છે. બ્રીજ પર પુરઝડપે જતાં વાહનો વચ્ચે અકસ્માતના બનાવો વધી ગયાં છે. બીજી તરફ નર્મદા મૈયા બ્રીજ પરથી છાસવારે લોકો આપઘાત કરવાના ઇરાદે નર્મદા નદીમાં ભુસકો મારતાં હોવાના બનાવો સામે આવી રહ્યાં છે. ત્યારે નર્મદા મૈયા બ્રીજ પરથી ભારદારી વાહનોની અવરજવર બંધ કરવા તેમજ પોલીસ દ્વારા બ્રીજ પર સતત પેટ્રોલિંગ કરવામાં આવે તેવી માંગ ઉઠી છે.અંકલેશ્વર અને ભરૂચ ને જોડાતા નર્મદા મૈયા બ્રિજ પર અચાનક રીક્ષા ઉભી થઇ જતાં કાર ધડાકાભેર ઘુસી જવા પામી હતી . 7 વ્યક્તિઓને ઇજા પહોંચી હતી. 108 ની મદદ થી ઇજાગ્રસ્તોને સારવાર અર્થે ખસેડાયો હતા. અકસ્માતના પગલે વાહન વ્યવહાર પર આંશિક અસર થઇ હતી.અંકલેશ્વરથી ભરૂચ તરફ જતી પેસેન્જર ભરેલી રીક્ષા નર્મદા મૈયા બ્રિજ પર એકાએક ઊભી કરી દેતા પાછળથી આવી રહેલ કાર રીક્ષા માં ધડાકાભેર ભટકાઈ ગયા હતા. કાર અને રીક્ષા વચ્ચે સર્જાયેલા અકસ્માતમાં રીક્ષા માં સવાર ડ્રાઈવર સહિત એક વૃધ્ધ તેમજ એક મહિલા અને એક નાની બાળકી ને પગ માં તેમજ માથાના ભાગે વાગેલ જ્યારે મોટરકાર માં સવાર 3 જેટલા લોકો નો આબાદ બચાવ મોટર કાર ચાલક ને માથાના ભાગે ઇજા થતા સારવાર અર્થે 108 એમ્બ્યુલન્સ સેવા બોલાવી તાત્કાલિક ભરૂચ સિવિલ હોસ્પિટલ ખાતે ચારેય ને ખસેડવામાં આવેલ છે. ઘટના અને પગલે વાહન વ્યવહાર ને આંશિક અસર થવા પામી હતી જેને ત્રાફિક જવાનો એ હળવો કર્યો હતો."

generated_summary = "નર્મદા મૈયા બ્રીજના નિર્માણ બાદ આ બ્રીજ અકસ્માત ઝોન તેમજ સુસાઇડ પોઇન્ટ બની ગયો હોય તેમ લાગીઅંકલેશ્વરથી ભરૂચ તરફ જતી પેસેન્જર ભરેલી રીક્ષા એકાએક ઊભી કરી દેતા કાર ધડાકાભેર ઘુસી જવા પામી હતી7 વ્યક્તિઓને ઇજા પહોંચી હતી, 108 ની મદદથી ઇજાગ્રસ્તોને સારવાર અર્થે ખસેડાયો હતા. અકસ્માતના પગલે વાહન વ્યવહાર પર આંશિક અસર થઇ હ"

# Print them
print("Generated Summary:\n", generated_summary)
print("Reference Summary:\n", reference_summary)

# ⚡ BERTScore calculation
# Using multilingual model because Gujarati is low-resource
P, R, F1 = score([generated_summary], [reference_summary], lang="gu", model_type="microsoft/mdeberta-v3-base")

print("\nBERTScore:")
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1 Score:  {F1.mean().item():.4f}")


Generated Summary:
 નર્મદા મૈયા બ્રીજના નિર્માણ બાદ આ બ્રીજ અકસ્માત ઝોન તેમજ સુસાઇડ પોઇન્ટ બની ગયો હોય તેમ લાગીઅંકલેશ્વરથી ભરૂચ તરફ જતી પેસેન્જર ભરેલી રીક્ષા એકાએક ઊભી કરી દેતા કાર ધડાકાભેર ઘુસી જવા પામી હતી7 વ્યક્તિઓને ઇજા પહોંચી હતી, 108 ની મદદથી ઇજાગ્રસ્તોને સારવાર અર્થે ખસેડાયો હતા. અકસ્માતના પગલે વાહન વ્યવહાર પર આંશિક અસર થઇ હ
Reference Summary:
 ભરૂચ અને અંક્લેશ્વરને જોડતો નર્મદા મૈયા બ્રીજ બે શહેર વચ્ચેનું અંતર ઘટાડવા સાથે ટ્રાફિકની સમસ્યામાંથી મુક્ત અપાવનાર મહત્વનો બ્રીજ સાબિત થયો છે. જોકે, નર્મદા મૈયા બ્રીજના નિર્માણ બાદ આ બ્રીજ અકસ્માત ઝોન તેમજ સુસાઇડ પોઇન્ટ બની ગયો હોય તેમ લાગી રહ્યું છે. બ્રીજ પર પુરઝડપે જતાં વાહનો વચ્ચે અકસ્માતના બનાવો વધી ગયાં છે. બીજી તરફ નર્મદા મૈયા બ્રીજ પરથી છાસવારે લોકો આપઘાત કરવાના ઇરાદે નર્મદા નદીમાં ભુસકો મારતાં હોવાના બનાવો સામે આવી રહ્યાં છે. ત્યારે નર્મદા મૈયા બ્રીજ પરથી ભારદારી વાહનોની અવરજવર બંધ કરવા તેમજ પોલીસ દ્વારા બ્રીજ પર સતત પેટ્રોલિંગ કરવામાં આવે તેવી માંગ ઉઠી છે.અંકલેશ્વર અને ભરૂચ ને જોડાતા નર્મદા મૈયા બ્રિજ પર અચાનક રીક્ષા ઉભી થઇ જતા